df = df.loc[np.where(df['reviewTime'].str.contains('2018'))[0]].reset_index(drop=True)

In [1]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('amz_data_prep_2018.csv', sep=',')#original
df

,reviewerID,asin,reviewTime,unixReviewTime,overall
0,A310S8GKL783R8,0001384198,2018-04-13,1523577600,5.0
1,ABO3HOSXIPOZY,0001384198,2018-04-11,1523404800,4.0
2,AU4Z1Q4AX9ILA,0001384198,2018-04-10,1523318400,5.0
3,A1TGZVJZK8SMBX,0001384198,2018-04-9,1523232000,5.0
4,A3CCP99A7Z7HHT,0001384198,2018-04-8,1523145600,3.0
...,...,...,...,...,...
4476622,A34QQHRRG65E08,B01HJF9W84,2018-01-25,1516838400,5.0
4476623,A2ASY291VN2H6F,B01HJGOOMW,2018-03-22,1521676800,2.0
4476624,A3TJ82VBTRW3O6,B01HJGOOMW,2018-03-10,1520640000,1.0
4476625,A10GEXRHH85UDI,B01HJF704M,2018-03-15,1521072000,1.0


In [4]:
df['ckt'] = df['asin'].apply(lambda x: x.isdigit())
df = df.drop(df[df['ckt'] == False].index)
df = df.drop(['ckt'], axis=1)
df = df.reset_index(drop=True)
df

,reviewerID,asin,reviewTime,unixReviewTime,overall
0,A310S8GKL783R8,0001384198,2018-04-13,1523577600,5.0
1,ABO3HOSXIPOZY,0001384198,2018-04-11,1523404800,4.0
2,AU4Z1Q4AX9ILA,0001384198,2018-04-10,1523318400,5.0
3,A1TGZVJZK8SMBX,0001384198,2018-04-9,1523232000,5.0
4,A3CCP99A7Z7HHT,0001384198,2018-04-8,1523145600,3.0
...,...,...,...,...,...
2467316,A3CCBASQJQZC4G,1609319745,2018-07-10,1531180800,5.0
2467317,A3POP3AO0D5GRV,1609319745,2018-04-2,1522627200,5.0
2467318,A33YTN2JLRR53E,9875851809,2018-08-30,1535587200,5.0
2467319,AW0FCQVRKDLPK,9966643451,2018-03-28,1522195200,4.0


In [5]:
user2idx = dict()
for idx, user in enumerate(df['reviewerID'].unique()):
    user2idx[user] = idx+1 

item2idx =dict()
for idx, item in enumerate(df['asin'].unique()):
    item2idx[item] = idx+1
df['reviewerID'] = df['reviewerID'].apply(lambda x:user2idx[x])
df['asin'] = df['asin'].apply(lambda x:item2idx[x])
df

,reviewerID,asin,reviewTime,unixReviewTime,overall
0,1,1,2018-04-13,1523577600,5.0
1,2,1,2018-04-11,1523404800,4.0
2,3,1,2018-04-10,1523318400,5.0
3,4,1,2018-04-9,1523232000,5.0
4,5,1,2018-04-8,1523145600,3.0
...,...,...,...,...,...
2467316,1009719,492764,2018-07-10,1531180800,5.0
2467317,721880,492764,2018-04-2,1522627200,5.0
2467318,1361087,492765,2018-08-30,1535587200,5.0
2467319,1361088,492766,2018-03-28,1522195200,4.0


In [6]:
sorted_df = df.sort_values(by=['reviewerID', 'unixReviewTime'])
sorted_df = sorted_df.reset_index(drop=True)
sorted_df

,reviewerID,asin,reviewTime,unixReviewTime,overall
0,1,1,2018-04-13,1523577600,5.0
1,2,172883,2018-04-10,1523318400,5.0
2,2,1,2018-04-11,1523404800,4.0
3,3,10089,2018-02-27,1519689600,4.0
4,3,1,2018-04-10,1523318400,5.0
...,...,...,...,...,...
2467316,1361085,492763,2018-08-16,1534377600,5.0
2467317,1361086,492764,2018-08-21,1534809600,5.0
2467318,1361087,492765,2018-08-30,1535587200,5.0
2467319,1361088,492766,2018-03-28,1522195200,4.0


In [7]:
new_df = pd.DataFrame(columns=['session_id', 'user_id', 'item_id', 'timeframe', 'eventdate', 'overall'])
new_df['session_id'] = sorted_df['reviewerID']
new_df['item_id'] = sorted_df['asin']
new_df['timeframe'] = sorted_df['unixReviewTime']
new_df['eventdate'] = sorted_df['reviewTime']
new_df['overall'] = sorted_df['overall']
new_df = new_df.reset_index(drop=True)

In [8]:
new_df

,session_id,user_id,item_id,timeframe,eventdate,overall
0,1,NaN,1,1523577600,2018-04-13,5.0
1,2,NaN,172883,1523318400,2018-04-10,5.0
2,2,NaN,1,1523404800,2018-04-11,4.0
3,3,NaN,10089,1519689600,2018-02-27,4.0
4,3,NaN,1,1523318400,2018-04-10,5.0
...,...,...,...,...,...,...
2467316,1361085,NaN,492763,1534377600,2018-08-16,5.0
2467317,1361086,NaN,492764,1534809600,2018-08-21,5.0
2467318,1361087,NaN,492765,1535587200,2018-08-30,5.0
2467319,1361088,NaN,492766,1522195200,2018-03-28,4.0


In [10]:
new_df = new_df[:40000]

In [11]:
new_df.to_csv('amz_data_prep_2018_book_40k.csv', sep=',', index=False)

In [2]:
book_2018 = pd.read_csv('amz_prep_book_overall_reflect+2018.csv', sep=';')
book_2018

,session_id,user_id,item_id,timeframe,eventdate
0,1,NaN,10053,1523318400,2018-04-10
1,1,NaN,146448,1523318400,2018-04-10
2,1,NaN,1,1523404800,2018-04-11
3,2,NaN,10246,1519689600,2018-02-27
4,2,NaN,1,1523318400,2018-04-10
...,...,...,...,...,...
1550670,358050,NaN,379020,1526342400,2018-05-15
1550671,358051,NaN,379236,1523491200,2018-04-12
1550672,358051,NaN,379237,1523491200,2018-04-12
1550673,358052,NaN,379384,1535673600,2018-08-31


In [3]:
book_2018_6 = book_2018.drop(book_2018[book_2018['timeframe'] >= 1530403200].index)

In [ ]:
book_2018_3 = book_2018.drop(book_2018[book_2018['timeframe'] >= 1522540800].index)

In [ ]:
book = df.loc[:51_311_620].reset_index(drop=True)
del df

In [3]:
book = book.loc[np.where(book['reviewTime'].str.contains('2018'))[0]].reset_index(drop=True)
book = book.loc[np.where(book['overall'] >= 3)[0]].reset_index(drop=True)

In [4]:
book = book.drop(index=np.where(book.duplicated('reviewerID', keep=False) == False)[0], axis=0).reset_index(drop=True)

In [5]:
book['reviewTime'] = book['reviewTime'].apply(lambda x: x.split(' ')[2]+'-'+x.split(' ')[0]+'-'+x.split(' ')[1][:-1])

In [6]:
user2idx = dict()
for idx, user in enumerate(book['reviewerID'].unique()):
    user2idx[user] = idx+1 

item2idx =dict()
for idx, item in enumerate(book['asin'].unique()):
    item2idx[item] = idx+1

In [7]:
book['reviewerID'] = book['reviewerID'].apply(lambda x:user2idx[x])
book['asin'] = book['asin'].apply(lambda x:item2idx[x])
sorted_book = book.sort_values(by=['reviewerID', 'reviewTime'])
del book

In [8]:
new_df = pd.DataFrame(columns=['session_id', 'user_id', 'item_id', 'timeframe', 'eventdate'])
new_df['session_id'] = sorted_book['reviewerID']
new_df['item_id'] = sorted_book['asin']
new_df['timeframe'] = sorted_book['unixReviewTime']
new_df['eventdate'] = sorted_book['reviewTime']
new_df = new_df.reset_index(drop=True)
del sorted_book

In [9]:
new_df

,session_id,user_id,item_id,timeframe,eventdate
0,1,NaN,10053,1523318400,2018-04-10
1,1,NaN,146448,1523318400,2018-04-10
2,1,NaN,1,1523404800,2018-04-11
3,2,NaN,10246,1519689600,2018-02-27
4,2,NaN,1,1523318400,2018-04-10
...,...,...,...,...,...
1550670,358050,NaN,379020,1526342400,2018-05-15
1550671,358051,NaN,379236,1523491200,2018-04-12
1550672,358051,NaN,379237,1523491200,2018-04-12
1550673,358052,NaN,379384,1535673600,2018-08-31


In [10]:
new_df.to_csv('amz_prep_book_overall_reflect+2018.csv', sep=';', index=False)

### for getting data

In [102]:
book = pd.read_csv('amz_prep_book+2018.csv', sep=';')
movi = pd.read_csv('amz_prep_movie+2018.csv', sep=';')
elec = pd.read_csv('amz_prep_elec+2018.csv', sep=';')

In [107]:
book = pd.read_csv('amz_prep_book_overall_reflect+2018.csv', sep=';')
movi = pd.read_csv('amz_prep_movie_overall_reflect+2018.csv', sep=';')
elec = pd.read_csv('amz_prep_elec_overall_reflect+2018.csv', sep=';')

In [108]:
partition = elec.loc[:40_000]

In [109]:
partition

,session_id,user_id,item_id,timeframe,eventdate
0,1,NaN,1,1522540800,2018-04-1
1,1,NaN,5714,1522540800,2018-04-1
2,1,NaN,8281,1522540800,2018-04-1
3,1,NaN,13858,1522540800,2018-04-1
4,1,NaN,16779,1522540800,2018-04-1
...,...,...,...,...,...
39996,9325,NaN,13743,1517011200,2018-01-27
39997,9326,NaN,641,1516924800,2018-01-26
39998,9326,NaN,1115,1516924800,2018-01-26
39999,9326,NaN,10773,1516924800,2018-01-26


In [105]:
partition['item_id'].nunique(), partition['session_id'].nunique()

(15915, 9375)

In [110]:
partition['item_id'].nunique(), partition['session_id'].nunique()

(15825, 9327)

In [106]:
partition.to_csv('amz_elec_40_000.csv', sep=';', index=False)

In [111]:
partition.to_csv('amz_elec_overall_reflect_40_000.csv', sep=';', index=False)

In [112]:
book = pd.read_csv('amz_book_40_000.csv', sep=';')
movi = pd.read_csv('amz_movi_40_000.csv', sep=';')
elec = pd.read_csv('amz_elec_40_000.csv', sep=';')

In [123]:
part = elec

In [124]:
s, scale = 0, part.session_id.nunique()
for _id in part.session_id.unique():
    s += len(np.where(part['session_id'] == _id)[0])
s/scale

4.266773333333333